<a href="https://colab.research.google.com/github/loosak/pysnippets/blob/master/Deep_Dive_1.1_Functional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐍 Python Deep Dive Notes 1.


author: Fred Baptiste


Variables, Functions and Functional Programming, Closures, Decorators, Modules and Packages

```
# `hello python`
```



# 2. A Quick Refresher

## Ternary Operators

In [ ]:
# Pythonic way 🐍 - Use a ternary operator ✅
a = 33
sign = "positive" if a > 0 else "negative"
sign

'positive'

In [ ]:
import platform
PRG = './dna.py'
RUN = f'python {PRG}' if platform.system() == 'Windows' else PRG
RUN

'./dna.py'

## `do` .. `while` loop

In [ ]:
while True:
  name = input("Enter name: ")
  if len(name) > 2 and name.isprintable() and name.isalpha():
    break
print(f"Hello {name}")

Enter name: jo
Enter name: joe
Hello joe


In [ ]:
a = 0
while a < 10:
  a += 1
  if a % 2 == 0:
    continue
  print(a)

1
3
5
7
9


## try ... except ... finally

In [ ]:
a = 10
b = 0
try:
    a/b
except ZeroDivisionError:
  print('division by 0')
finally:
  print('this always executes')


division by 0
this always executes


An iterable is an object capable returning values one at a time

In [ ]:
for i in range(5):
  if i == 3:
    continue
  print(i, end=' ')

print('\n', '-'*10)

for i in range(5):
  if i == 3:
    break
  print(i, end=' ')

0 1 2 4 
 ----------
0 1 2 

In [ ]:
for i, c in enumerate('hello'):
  print(i, c)

0 h
1 e
2 l
3 l
4 o


## Classes

In [ ]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        
    def area(self):
        return self.width * self.height
    
    def perimeter(self):
        return 2 * (self.width + self.height)
    
    def __str__(self):
        return f"Rectangle (width={self.width}, height={self.height})"
    
    def __repr__(self):
        return f"Rectangle ({self.width}, {self.height})"

In [ ]:
r1 = Rectangle(10, 20)
print(r1)  # uses __str__

Rectangle (width=10, height=20)


In [ ]:
r1

Rectangle (10, 20)

That's because here Python is not converting `r1` to a string, but instead looking for a string *representation* of the object. It is looking for the `__repr__` method (which we'll come back to later).

In [ ]:
r1 = Rectangle(10, 20)
r2 = Rectangle(10, 20)
r1 == r2

False

In [ ]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        
    def area(self):
        return self.width * self.height
    
    def perimeter(self):
        return 2 * (self.width + self.height)
    
    def __str__(self):
        return f"Rectangle (width={self.width}, height={self.height})"
    
    def __repr__(self):
        return f"Rectangle ({self.width}, {self.height})"
    
    def __eq__(self, other: object) -> bool:
        print(f'self={self}, other={other}')
        if isinstance(other, Rectangle):
          return (self.width, self.height) == (other.width, other.height)
        else:
          return False

In [ ]:
r1 = Rectangle(10, 20)
r2 = Rectangle(10, 20)
r1 is r2, r1 == r2

self=Rectangle (width=10, height=20), other=Rectangle (width=10, height=20)


(False, True)

What about `<`, `>`, `<=`, etc.?

Again, Python has special methods we can use to provide that functionality.

These are methods such as `__lt__`, `__gt__`, `__le__`, etc.

In [ ]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        
    def area(self):
        return self.width * self.height
    
    def perimeter(self):
        return 2 * (self.width + self.height)
    
    def __str__(self):
        return f"Rectangle (width={self.width}, height={self.height})"
    
    def __repr__(self):
        return f"Rectangle ({self.width}, {self.height})"
    
    def __eq__(self, other: object) -> bool:
        if isinstance(other, Rectangle):
          return (self.width, self.height) == (other.width, other.height)
        else:
          return False

    def __lt__(self, other: object) -> float:
        if isinstance(other, Rectangle):
            return self.area() < other.area()
        else:
            return NotImplemented
            


In [ ]:
r1 = Rectangle(100, 200)
r2 = Rectangle(10, 20)
r1 < r2, r2 > r1

(False, False)

How did that work? We did not define a `__gt__` method.

Well, Python cleverly decided that since `r1 > r2` was not implemented, it would give 

`r2 < r1` 

a try. And since, `__lt__` **is** defined, it worked!

Of course, `<=` is not going to magically work!

In [ ]:
r1 <= r2

TypeError: ignored

### property getters and setters:
In Pytyhon we can use some special **decorators** (more on those later) to encapsulate our property getters and setters:

In [ ]:
class Rectangle:
    def __init__(self, width, height):
        self._width = width
        self._height = height
    
    def __repr__(self):
        return f"Rectangle ({self.width}, {self.height})"
        
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self, width):
        if width <= 0:
            raise ValueError('Width must be positive.')
        self._width = width
    
    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self, height):
        if height <= 0:
            raise ValueError('Height must be positive.')
        self._height = height

In [ ]:
r1 = Rectangle(10, 20)
r1.width

10

In [ ]:
r1 = Rectangle(-1, 11)

# 3. Variables and Memory


## Variables are memory references

In [ ]:
a = 10
ptr = id(a)
ptr, hex(ptr)

(94470284819232, '0x55eb93c4bb20')

## Reference counting

passing var to `sys.getrefcount()` creates extra reference!

In [ ]:
import sys

def f1():
  ...

my_func1 = f1
print(sys.getrefcount(my_func1))
my_func2 = my_func1
print(sys.getrefcount(my_func1))
id(f1), id(my_func1), id(my_func2)

3
4


(140551679434832, 140551679434832, 140551679434832)

In [ ]:
import ctypes
var1 = 10
address = id(var1)
ctypes.c_long.from_address(id(var1)).value

572

In [ ]:
import sys

a = [1, 2, 3]
id(a)

140551473220976

In [ ]:
sys.getrefcount(a)

3

In [ ]:
import ctypes

def ref_count(address: int) -> int:
  return ctypes.c_long.from_address(address).value

ref_count(id(a))

6

In [ ]:
 import gc

len(gc.get_objects())

148762

## Circular reference

In [ ]:
import ctypes
import gc

def ref_count(address: int) -> int:
  return ctypes.c_long.from_address(address).value

def object_by_id(object_id):
  for obj in gc.get_objects():
    if id(obj) == object_id:
      return f"Object id {hex(object_id)} exists"
  return f"Object id {hex(object_id)} not found"
    
class A:
  def __init__(self):
    self.b = B(self)
    print(f'A: self: {hex(id(self))}, b: {hex(id(self.b))}')

class B:
  def __init__(self, a):
    self.a = a
    print(f'B: self: {hex(id(self))}, a: {hex(id(self.a))}')

gc.disable()
my_var = A()

B: self: 0x7fb4a7e05810, a: 0x7fb4a7e05a10
A: self: 0x7fb4a7e05a10, b: 0x7fb4a7e05810


In [ ]:
hex(id(my_var)) == hex(id(my_var.b.a))

True

In [ ]:
hex(id(my_var.b))

'0x7fb4a7e05810'

In [ ]:
a_id = id(my_var)
b_id = id(my_var.b)
hex(a_id), hex(b_id)

('0x7fb4a7e05a10', '0x7fb4a7e05810')

In [ ]:
ref_count(a_id), ref_count(b_id)

(3, 1)

In [ ]:
my_var = None
ref_count(a_id), ref_count(b_id)

(2, 1)

In [ ]:
def object_by_id(object_id):
  for obj in gc.get_objects():
    if id(obj) == object_id:
      return f"Object id {hex(object_id)} exists"
  return f"Object id {hex(object_id)} not found"

object_by_id(a_id), object_by_id(b_id)

('Object id 0x7fb4a7e05a10 exists', 'Object id 0x7fb4a7e05810 exists')

In [ ]:
 # run the garbege collector
 gc.collect()
 object_by_id(a_id), object_by_id(b_id)

('Object id 0x7fb4a7e05a10 not found', 'Object id 0x7fb4a7e05810 not found')

In [ ]:
ref_count(a_id), ref_count(b_id)

(0, 0)

## var re-assigment
in fact, the value inside the object, can *never* be changed


In [ ]:
my_var = 10
print(hex(id(my_var)))

my_var += my_var
print(hex(id(my_var)))

my_var = float(my_var)
print(hex(id(my_var)))

a = 10
b = 10
hex(id(a)), hex(id(b)) # same as my_var = 10

0x559e4b171b20
0x559e4b171c60
0x7fb4a7bb5990


('0x559e4b171b20', '0x559e4b171b20')

### Object Mutability
- *modifying internal object state*: changing the data inside the object

- mutable: internal object state can be changed

- inmutable: internal object state can be changed
 - numbers (int, float, bool, complex)
 - str
 - tuple
 - frozen sets
 - class (user def)

 - mutable:
  - list
  - set
  - dict
  - class (user def)

Warning: Immutability does mean frozen!!

In [ ]:
l1 = [1, 2]
l2 = [3, 4]
t = l1, l2
type(t)

tuple

In [ ]:
t.append([5, 6])

AttributeError: ignored

In [ ]:
l1.append(3)
t

([1, 2, 3], [3, 4])

In [ ]:
l2 += [5, 6]
t

([1, 2, 3], [3, 4, 5, 6])

In [ ]:
t[0] += [1]

TypeError: ignored

## Functional Arguments and Mutability

mutable objects are not safe from sideeffects

In [ ]:
def process(s):
  print(f'INITIAL s # = {id(s)}')
  s += ' world'
  print(f'FINAL   s # = {id(s)}')
  return s

v = 'hello'
process(v), id(v)

INITIAL s # = 140025362302128
FINAL   s # = 140025150302320


('hello world', 140025362302128)

In [ ]:
def process(lst):
  print(f'INITIAL lst # = {id(lst)}')
  lst.append(100)
  print(f'FINAL   lst # = {id(lst)}')
l = [1, 2, 3]

process(l), id(l), l

INITIAL lst # = 140025150333648
FINAL   lst # = 140025150333648


(None, 140025150333648, [1, 2, 3, 100])

In [ ]:
def process(t):
  t[0].append(3)

t = ([1, 2], 'A')

process(t), t

(None, ([1, 2, 3], 'A'))

### Mutable Default Arguments
Python supports default values for function parameters... There is a danger associated with this if the default value is of a mutable type. For example, consider specifying an empty list as a default value. If the list is modified, the default value is modified *as well*. In most cases, this is not intended. To avoid it, we can set the default value to `None`. If no value is passed during the function call, we can ensure an empty list is created.

In [ ]:
# Mutable default arguments 💩:  Wrong way  ❌
def append_element(elem, L=[]):
    L.append(elem)
    return L

L1 = append_element(21) 
L2 = append_element(42)
L1, L2

([21, 42], [21, 42])

In [ ]:
# Correct way 🔥: Use None ✅
def better_append(elem, L=None):
    if L is None:
        L = []
    L.append(elem)
    return L

L1 = L2 = []
L1 = append_element(21) 
L2 = append_element(42)
L1, L2

([21, 42, 21, 42, 21, 42, 21, 42, 21, 42],
 [21, 42, 21, 42, 21, 42, 21, 42, 21, 42])

## Shared reference

In [ ]:
a = 10
b = a

# with mutable objects python never make shared reference
a = [1, 2, 3]
b = a
b.append(100)

## Var equaility
* mem adress: `is`
* oject internal state:  `==`

In [ ]:
a = 10
b = 10
a is b

True

In [ ]:
a = [1, 2, 3]
b = [1, 2, 3]
a is b, a == b

(False, True)

In [ ]:
a = 10
b = 10 + 0j
print(a is b, a == b)
a = complex(a)
a is b, a == b

False True


(False, True)

### `None` object
* empty value null pointer
* a real object managed by memory manager (shared reference) `is None`

In [ ]:
id(None)

94821412023744

In [ ]:
a = None
b = None
c = None
a is b, a is c, a is None, c is None, type(c), id(c) == id(a), id(c) == id(None)

(True, True, True, True, NoneType, True, True)

## Everything is objects (instace of calsses)
* have memory add

Any object (including function) can be:
- assigned to var
- passed as parameter to a function
- returned from a function

In [ ]:
def f1():
  ...

f2 = f1
id(f1), id(f2), f1 is f2

(139821540817808, 139821540817808, True)

In [ ]:
a = 10
b = int(10)
id(a), id(b)

(94821412629280, 94821412629280)

In [ ]:
def square(a):
  return a ** 2

def cube(a):
  return a ** 3

def select_fce(f_id):
  if f_id == 1:
    return square
  else:
    return cube

f = select_fce(1)
f is square, f(2)

(True, 4)

In [ ]:
select_fce(2)(3)

27

In [ ]:
def exec_fn(fn, param):
  return fn(param)

exec_fn(cube, 2)

8

## Python Optimization

https://wiki.python.org/moin/PythonImplementations

* [CPython](https://github.com/python/cpython) 
    Reference implementation in C
* [pyston](https://github.com/pyston/pyston)
    fork of CPython 3.8.12 with additional optimizations for 30% performance 
* [IronPython](https://github.com/IronLanguages/ironpython3)
    Cross-platform support the .NET Core, .NET and Mono runtimes.
*  [PyPy]()
    replacement for CPython with JIT compiler
    ```
    brew install pypy3
    pypy3 -> PyPy 7.3.8 with GCC Apple LLVM 13.0.0 (clang-1300.0.29.30
    pip_pypy3 list
    pypy3 -m pip install numpy
    ```
* [rustpython](https://github.com/RustPython/RustPython)
    Full Python 3 environment entirely in Rust can be used from Rust or compiled to WebAssembly.
    ```
    cargo install rustpython
    or
    cargo install --git https://github.com/RustPython/RustPython
    or 
    conda install rustpython -c conda-forge

    wapm install rustpython/rustpython
    curl https://get.wasmer.io -sSfL | sh
    ```
* [Jython](https://github.com/jython/jython/)
  * [JythonBook](https://jython.readthedocs.io/en/latest/)

```
wget https://repo1.maven.org/maven2/org/python/jython-standalone/2.7.2/jython-standalone-2.7.2.jar
java -jar jython-standalone-2.7.2.jar
import sys
>>> sys.version
'2.7.2 (v2.7.2:925a3cc3b49d, Mar 21 2020, 10:03:58)\n[Java HotSpot(TM) 64-Bit Server VM (Oracle Corporation)]'

---
import java
from java import awt

def exit(e): java.lang.System.exit(0)

frame = awt.Frame('AWT Example', visible=1)
button = awt.Button('Close Me!', actionPerformed=exit)
frame.add(button, 'Center')
frame.pack()
---
java -jar jython-standalone-2.7.2.jar awt.py
```

* [Pyodide](https://github.com/pyodide/pyodide)
  a port of CPython to WebAssembly/Emscripten
  When used inside a browser, Python has full access to the Web APIs
  https://github.com/alexmojaki/futurecoder


### Interning
reusing object on-demand
CPython pre-loades(caches) a global list of int of range(-5, 256)
Singleton


In [ ]:
a = 257
b = 257
a is b, id(a), id(b)

(False, 139821541124112, 139821541123760)

In [ ]:
gi = (i for i in range(-5, 256))
a = next(gi)
b = next(gi)
print(a,b, id(a), id(b))
c = -5
id(c)

-5 -4 94821412628800 94821412628832


94821412628800

### Strings interned
* identifiers
* literals which looks like identifiers `hello_world`
* `is` is much faster than `==` (char by char)

In [ ]:
a = 'hello_world'
b = 'hello_world'
id(a), id(b), a is b

(139821541388016, 139821541388016, True)

In [ ]:
a = 'the quick brown fox'
b = 'the quick brown fox'
a is b

False

In [ ]:
import sys
a = sys.intern('the quick brown fox')
b = sys.intern('the quick brown fox')
a is b

True

In [ ]:
import sys
import time

def compare_using_equals(n):
  a = 'the quick brown fox' * 200
  b = 'the quick brown fox' * 200
  for i in range(n):
    if a == b:
      pass

def compare_using_interning(n):
  a = sys.intern('the quick brown fox' * 200)
  b = sys.intern('the quick brown fox' * 200)
  for i in range(n):
    if a is b:
      pass

start = time.perf_counter()
compare_using_equals(10_000_000_000)
end = time.perf_counter()
end-start

0.5551720000003115

In [ ]:
start = time.perf_counter()
compare_using_interning(10_000_000_000)
end = time.perf_counter()
end-start

472.58671015500295


## Membership tests

* const imutable
lists -> tuples
sets -> frozensets

set membership is faster than list/tuple membership



In [ ]:
e = 2
e in [1, 2, 3], e in (1, 2, 3)

(True, True)

In [ ]:
e in {1, 2, 3}

True

In [ ]:

f1.__code__.co_consts

(None,
 1440,
 (1, 2, 1, 2, 1, 2, 1, 2, 1, 2),
 'abcabcabc',
 'ababababababababababab',
 'the quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown fox',
 'a',
 'b',
 3)

In [ ]:
def f2(e):
  if e in [1,2,3]:
    pass

f2.__code__.co_consts

(None, (1, 2, 3))

In [ ]:
import string
import time
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [ ]:
ch_l = list(string.ascii_letters)
ch_t = tuple(string.ascii_letters)
ch_s = set(string.ascii_letters)
print(ch_s)


{'O', 'X', 'N', 'Z', 'K', 'e', 'I', 'M', 'G', 'P', 'z', 't', 'Q', 'J', 'L', 'a', 'F', 'h', 'Y', 'U', 'w', 'u', 'y', 'l', 'q', 'H', 'v', 'j', 'k', 'R', 'T', 'x', 'i', 'g', 'E', 'b', 'A', 'n', 'B', 'o', 'r', 'C', 'W', 'd', 'c', 'V', 'p', 'D', 's', 'f', 'S', 'm'}


In [ ]:
def membership_test(n, container):
  for i in range(n):
    if 'x' in container:
      pass

start = time.perf_counter()
membership_test(10_000_000, ch_l)
end = time.perf_counter()
end-start

4.944840404000047

In [ ]:
start = time.perf_counter()
membership_test(10_000_000, ch_t)
end = time.perf_counter()
end-start

4.826176579000048

In [ ]:
start = time.perf_counter()
membership_test(10_000_000, ch_s)
end = time.perf_counter()
end-start

0.5190618799999811

In [ ]:
import string
import time


def membership_test(n, container):
  for i in range(n):
    if 'x' in container:
      pass


In [ ]:
%%timeit

container = tuple(string.ascii_letters)
start = time.process_time()
membership_test(10_000_000, container)
print(time.process_time()-start)

4.880783031
4.808397572999999
4.750707510000002
4.782601762999999
4.774836266999998
4.776945073
1 loop, best of 5: 4.76 s per loop


In [ ]:
%%timeit

container = set(string.ascii_letters)
start = time.process_time()
membership_test(10_000_000, container)
print(time.process_time()-start)

0.5352344369999997
0.5426121320000021
0.5338068860000078
0.5397688329999966
0.5284100149999915
0.5395970550000015
1 loop, best of 5: 528 ms per loop


In [ ]:
%%bash
ls -la .config

total 44
drwxr-xr-x 1 root root 4096 Apr  8 13:31 .
drwxr-xr-x 1 root root 4096 Apr  8 13:32 ..
-rw-r--r-- 1 root root    7 Apr  8 13:31 active_config
-rw-r--r-- 1 root root    0 Apr  8 13:32 config_sentinel
drwxr-xr-x 2 root root 4096 Apr  8 13:31 configurations
-rw-r--r-- 1 root root  101 Apr 12 13:10 .feature_flags_config.yaml
-rw------- 1 root root    5 Jan  1  2040 gce
-rw-r--r-- 1 root root    3 Apr  8 13:31 .last_opt_in_prompt.yaml
-rw-r--r-- 1 root root   37 Apr  8 13:31 .last_survey_prompt.yaml
-rw-r--r-- 1 root root  135 Apr  8 13:31 .last_update_check.json
drwxr-xr-x 3 root root 4096 Apr  8 13:31 logs
-rw-r--r-- 1 root root   32 Apr  8 13:31 .metricsUUID


# ---

In [ ]:
import ctypes
libc = ctypes.CDLL("libc.so.6") # /usr/lib/libSystem.dylib -> man 3 intro
message_string = "Hello world!\n" 
libc.printf("Testing: %s", message_string)

1

In [ ]:
#@title Example form fields
#@markdown Forms support many types of fields.

no_type_checking = ''  #@param
string_type = 'example'  #@param {type: "string"}
slider_value = 142  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---


In [ ]:
string_type = 'example' #@param {type: "string"}

In [ ]:
import re
re_examples = [
  not re.match("a", "cat"),
  re.search("a", "cat"),
  not re.search("c", "dog"),
  3 == len(re.split("[ab]", "carbs")),
  "R-D-" == re.sub("[0-9]", "-", "R2D2") # Replace digits with dashes. 
]
assert all(re_examples), "all the regex examples should be True"

In [ ]:
cat:str = "🐱"
cat

'🐱'

# List Comprehensions

To write a list comprehension, start with the expression you would normally pass to the `append` method. From there, write the for loop condition immediately after the initial expression. Lastly, put everything inside a pair of square brackets. Comprehensions can be used with dictionaries, sets, and generators, however, try to avoid them with complex expressions. Readability is key.

In [ ]:
# OK version 🤔 - For loop and append ❌ 
squares = []
for num in range(12):
    squares.append(num ** 2)

# Pythonic version 🐍: Use a list comprehension ✅
squares = [num ** 2 for num in range(12)]
squares

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

## Bonus Tip 💡: You can also use dictionary, set, and generator comprehensions

In [ ]:
squares_dict = {num: num ** 2 for num in range(12)} # dictionary
squares_set = {num ** 2 for num in range(12)}       # set
squares_gen = (num ** 2 for num in range(12))       # generator

squares_dict, squares_set, list(squares_gen)

({0: 0,
  1: 1,
  2: 4,
  3: 9,
  4: 16,
  5: 25,
  6: 36,
  7: 49,
  8: 64,
  9: 81,
  10: 100,
  11: 121},
 {0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121},
 [0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121])

# Generators
**Generators** are a powerful tool to save memory and improve performance. In general, they yield one value at a time and can be iterated over multiple times. Let’s imagine we’re interested in the sum of the first 42 000 natural numbers. We could use a list comprehension to compute the values and call the built-in `sum` function. Building a list requires 351064 bytes. Using a generator reduces this value to 112 bytes. That’s pretty awesome 🔥

In [ ]:
import sys

# Inefficent way 💩: Using a list ❌
l = [item for item in range(42_000)]
sum(l), f'{sys.getsizeof(l)} bytes'

(881979000, '361296 bytes')

In [ ]:
# Efficient way 🔥: Use a generator ✅
g = (item for item in range(42_000))
sum(g), f'{sys.getsizeof(g)} bytes'

(881979000, '128 bytes')